

# 

#  Numba Lab1: Numba For CUDA GPU
---

## Learning Objectives
- **The goal of this lab is to:**
    -   enable you to quickly start using Numba (beginner to advanced level)
    -   teach you to apply the concepts of CUDA GPU programming to HPC field(s); and
    -   show you how to achieve computational speedup on GPUs to maximize the throughput of your HPC implementation.


Before we begin, let's execute the cell below to display information about the CUDA driver and GPUs running on the server by running the `nvidia-smi` command. To do this, execute the cell block below by clicking on it with your mouse, and pressing Ctrl-Enter, or pressing the play button in the toolbar above. You should see some output returned below the grey cell.

In [ ]:
!nvidia-smi

     
##  Introduction

**Numba** is a just-in-time compiler for python that converts python functions into optimized machine code at runtime. In other words, user-defined functions written in python would be run at native machine code speed. For example, a programmer can delegate functions that are computationally intensive (especially those with consecutive nested loops and arrays) within his/her code to Numba execution and gain speed up. This is achievable by placing Numba decorator at the top of a user-define function. A Numba decorator determines how a function would be compiled, and more on it would be explained later in the notebook. Numba has huge support for NumPy library and also enables parallel programming on `CPU (multicore) and GPU (via CUDA API binding)`, thus making execution on NumPy arrays faster. Our focus would be using `Numba for CUDA GPU`, therefore, parallel programming concept would be described from CUDA C kernel perspective. The rest of this notebook would include frequently use terms like **Host** (this refers to a CPU), **Device** (means a GPU), and **Kernel** (a user-defined function that runs on the GPU with Numba decorator specified at the top).

## Memory Architecture

<p>When written codes run on the Device (GPU), execution is shared amongst threads and blocks memory space. The execution could be mapped to thousands of threads modelled in blocks and grids form. This idea is illustrated in figure 1.0 with a view that a thread can be seen as a single unit on the Device.  A thread block (also known as a block) is as collection of threads that can communicate, while a collection of these blocks is referred to as a Grid. In several devices the maximum number of threads within a thread block could be 1024 and 65535 blocks within a grid.</p>

 <img src="../images/thread_blocks.JPG" height="720px" width="640px"/> 
 <div style="margin-left:300px">Figure 1.0. Thread, block, and grid concept </div> 
 
 <p>As shown in figure 2.0, the GPU memory space is hierarchically arranged into share memory, local memory, global memory, constant memory, and texture memory. Currently, Numba does not implement texture memory CUDA features. Within a block, each thread has its own local memory and register and does communicate with other threads using the shared memory. </p>
 
 <img src="../images/memory_architecture.png" height="512px" width="600px"/> 
 <div style="margin-left:370px">Figure 2.0. Memory Architecture</div> 
 
 ***Image source***: <i>Bhaumik Vaidya, Hands-On GPU-Accelerated Computer Vision with OpenCV and CUDA, Packt Publishing, 2018</i>.
 

## Numba CUDA Kernel

This section highlights steps in writing your first CUDA Kernel in Numba. The steps would be illustrated using a simple task as follows:

**Task 1**: Add two arrays A and B together and stores the result in array C. The size of A and B is 10,000.

#### Step 1: 
 - First, `import numba.cuda as cuda` library at the top of your notebook in order to access `cuda.jit`. 
 - Next, write an empty python function and specify `@cuda.jit` at the top of the function. An example is given below:


```python
import numba.cuda as cuda
@cuda.jit
def <function_name>(<arguments>):
    #...code body ...
```

 - **Write code body**: To successfully write the kernel code body, we must understand that computation within CUDA kernels execute in thread blocks and grids such that values in input array are accessed using thread id as index. Therefore, it is important to uniquely identify distinct threads. A simple illustration on how to estimate thread id(s) is given in figure 3.0 using four blocks of threads stacked over each other to form a matrix in row column arrangement. Thread id(s) are calculated in x-dimension (ideally thread block are in x,y,z diamensions) by viewing the stack as a single row and then apply the statement below:

```python
tid = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
```
<img src="../images/thread_position.png" height="400px" width="400px"/>
<div style="margin-left:300px">Figure 3.0 Estimating thread id for threads in green and orange</div>

Now that we know how to compute thread ids, we can proceed to write the kernel body code as follows:  

```python
import numba.cuda as cuda
N = 10000 #initialize array size
@cuda.jit
def Add_arrays(d_A, d_B, d_C):
	tid = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
    if tid < N: #ensuring that index is not out of bound
        d_C[tid] = d_A[tid] + d_B[tid]

```

Note that kernel function does not return value through variable, it is stream/copy back from the Device to Host.

#### Step 2:

- **Write the Host code**- The first thing to do is to initialize your input arrays as follows:
 
```python
import numpy as np
h_A = np.arange(N, dtype=np.int32)
h_B = np.arange(N, dtype=np.int32)
h_C = np.zeros(N, dtype=np.int32) #initialize zero filled array
````
Next is data transfer by copying data (input array) from the Host to the Device using `cuda.to_device()` function. 
 
```python
d_A = cuda.to_device(h_A)
d_B = cuda.to_device(h_B)
d_C = cuda.to_device(h_C)

```
Note that kernel written in Numba automatically has direct access to NumPy arrays residing on the Host and its implies not explicitly stating ```cuda.to_device()``` for a NumPy array to be visible on the Device. Consequently, unneeded data from the Device may be visible(streamed) to Host consuming memory space unnecessarily.

#### Step 3: 

With the existence of on the Device and visibility to the Kernel, the next step would be to call the kernel function from the Host. But before that, it is important to initialize the number of threads that would make up a single block (thread block) so that number of blocks in a grid required to execute `Add_array` Kernel can be estimated. In Numba, Kernel calls has a definition pattern stated as follows:

`<kernel_name> [ <num_of_blocks_per_grid>, <num_of_threads_per_block>] (<arguments>)`

The total number of threads required is equivalent to the size of our array which is `10,000`, therefore:

```python 
num_of_threads_per_block = 256 # this has not exceed the limit i.e < 1024 
```

Then, `num_of_blocks_per_grid` can be estimated as:

```python 
num_of_blocks_per_grid = math.ceil(N / num_of_threads_per_block)
```

Subsequently, Add_arrays Kernel function can be called this way:

`Add_arrays[num_of_blocks_per_grid , num_of_threads_per_block](d_A, d_B, d_C)`

#### Step 4:

Copy result from Device to Host using `copy_to_host()` function, thus:

```python
h_C = d_C.copy_to_host()
````
Thereafter, the value of `h_c` is printed to the console. You can run the entire code in the below.


In [ ]:
import math
import numba.cuda as cuda
import numpy as np

N = 10000 #initialize array size
#kernel function
@cuda.jit
def Add_arrays(d_A, d_B, d_C):
    tid = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
    if tid < N: #ensuring that index is not out of bound
        d_C[tid] = d_A[tid] + d_B[tid]
        
#input data initialzed on the Host
h_A = np.arange(N, dtype=np.int32)
h_B = np.arange(N, dtype=np.int32)
h_C = np.zeros(N, dtype=np.int32) # initialize zero filled array

#input data copied to Device
d_A = cuda.to_device(h_A)
d_B = cuda.to_device(h_B)
d_C = cuda.to_device(h_C)

#set block and grid size and call kernel
num_of_threads_per_block = 256 # this has no exceed the limit i.e < 1024
num_of_blocks_per_grid = math.ceil(N / num_of_threads_per_block)
Add_arrays[num_of_blocks_per_grid , num_of_threads_per_block](d_A, d_B, d_C)

#copy result back to Host
h_C = d_C.copy_to_host()
print("h_C :", h_C)

####################################################################
#expected output: h_C : [    0     2     4 ... 19994 19996 19998]


---
**Exercise 1**: Follow the steps highlighted above and write a Numba program (Host & Device code) that adds two arrays and stores the result in a third array. The size of each array is 500,000. Execute this task in the cell below:

---

In [ ]:
import math
#import library
#import library

N = 500000 #initialize array size

#kernel function
def Addition_kernel(A, B, C):
    #write kernel code body


########## Host code body #########

#input data initialzed on the Host



#input data copied to Device



#set block and grid size and call kernel



#copy result back to Host



################### expected output #################################
# [     0      2      4 ... 999994 999996 999998]
######################################################################

## Thread Reuse

It is possible to specify a few numbers of thread for a large data in a way that threads are reused to complete the computation of the entire data. This is one of the approaches used when data to be computed is larger than the maximum number of threads available in the device memory. The statement below is used in a `while loop` to achieve such purpose: 

```python 
tid += cuda.blockDim.x * cuda.gridDim.x
```
The sample code given below illustrates thread reuse using task 1 as a case study. In the example, number of blocks per grid is set to 1 on purpose to show the possibility of this approach. Therefore, a single block of thread having 256 threads would be reused to compute addition operation on two arrays, each of size 10,000. 

```python
import numba.cuda as cuda
import numpy as np

N = 10000 #initialize array size
#kernel function
@cuda.jit
def arrayAdd(d_A, d_B, d_C):
   tid = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
   while tid < N:
      array_out[tid] = array_A[tid] + array_B[tid]
      tid += cuda.blockDim.x * cuda.gridDim.x
        
################################################
#set block and grid size and call kernel
num_of_threads_per_block = 256
num_of_blocks_per_grid = 1
arrayAdd[num_of_blocks_per_grid, threadsperblock](d_A, d_B, d_C)

print(array_out)

#output: [    0     2     4 ... 19994 19996 19998]

```

## 2-Dimensional Array

In this section, focus would be on performing simple calculation with 2D arrays. In two different approaches, let’s consider multiplication of `4x4` arrays A and B. In the first approach, each array is split into `2x2` segments in a way that threads per block and blocks per grid would be `2x2` respectively.  In the second approach, each array would fit into a thread block as `4x4` and block per grid would be `1x1(1 grid)`. This process is exemplified in figure 4.0 using small array size for ease of understanding. It is assumed that the mathematical process of multiply two matrixes is already known, however, a glimpse on process is shown figure 5.0.

<img src="../images/2d_array.png" height="512px" width="600px"/>
<div style="margin-left:300px">Figure 4.0 2D array device fitting logic</div>
<br/>
<img src="../images/2d_col_mult.png" height="400px" width="400px"/>
<div style="margin-left:330px">Figure 5.0 Matrix multiplication logic</div>

**Implementation:** The 4 basic steps that were previously explained would be followed.

#### Step 1
```python
import numba.cuda as cuda
import numpy as np
import math

N = 4 #initialize array size

@cuda.jit()
def MatrixMul2D(d_A, d_B, d_C):
   row, col = cuda.grid(2)
   if row < d_C.shape[0] and col < d_C.shape[1]:
      for k in range(N):
         d_C[row][col]+= d_A[row][k] * d_B[k][col]
```
#### step 2
```python
#input data initialzed on the Host 
h_A = np.array([[0,0,0,0],[1,1,1,1],[2,2,2,2],[3,3,3,3]], dtype=np.int32)
h_B = np.array([[0,1,2,3],[0,1,2,3],[0,1,2,3],[0,1,2,3]], dtype=np.int32)
h_C = np.zeros(N*N, dtype=np.int32).reshape(N, N)

#input data copied to Device
d_A = cuda.to_device(h_A)
d_B = cuda.to_device(h_B)
d_C = cuda.to_device(h_C)

```
#### step 3
```python
#set block and grid size and call kernel
num_of_threads_per_block  = (2,2)
num_of_blocks_per_grid_x  = (math.ceil( N / num_of_threads_per_block[0]))
num_of_blocks_per_grid_y  = (math.ceil( N / num_of_threads_per_block[1]))
num_of_blocks_per_grid    = (num_of_blocks_per_grid_x , num_of_blocks_per_grid_y)

MatrixMul2D[num_of_blocks_per_grid, num_of_threads_per_block](d_A, d_B, d_C)
```
#### Step 4
```python
h_C = d_C.copy_to_host()

print("h_A:\n {}\n".format(h_A))
print("h_B:\n {}\n".format(h_B))
print("h_C:\n {}".format(h_C))

```
---
**Exercise 2**: Write a Numba program (Host & Device) that multiply two matrixes of `225 x 225` dimensions. Part of the code has been written for you in the cell below and you are to complete the rest. In the solution, the intention is to split each matrix into `25x25` segments such that threads per block and blocks per grid would be `25x25` respectively.

---

In [ ]:
import math
import numpy 
#import library

N = 225 #initialize array size

#kernel function
def MatrixMul2D(d_A, d_B, d_C):
  x, y = cuda.grid(2)
  #complete kernel code


#input data initialized on the Host
h_A = np.arange((N*N), dtype=np.int32).reshape(N,N)
h_B = np.arange((N*N), dtype=np.int32).reshape(N,N)
h_C = np.zeros((N*N), dtype=np.int32).reshape(N,N)


#input data copied to Device



#set block and grid size and call kernel
num_of_threads_per_block = (25,25)



#copy result back to Host




###################### expected output #########################################

#[[  848610000   848635200   848660400 ...   854204400   854229600 854254800]
# [ 2124360000  2124435825  2124511650 ...  2141193150  2141268975 2141344800]
# [ -894857296  -894730846  -894604396 ...  -866785396  -866658946 -866532496]
# ...
# [  597268464   608532414   619796364 ... -1197101932 -1185837982 -1174574032]
# [ 1873018464  1884333039  1895647614 ...    89886818   101201393 112515968]
# [-1146198832 -1134833632 -1123468432 ...  1376875568  1388240768 1399605968]]
################################################################################


## CUDA Universal Functions (Ufuncs) and Device Function

Ufuncs are NumPy functions used to implement vectorization on ndarray. Vectorization supports broadcasting mechanism that eliminates the use of loop(s) when operating on ndarrays and as a result, execution is faster. Numba implements CUDA Ufuncs using `@vectorize()` decorator as shown in figure 6.0. Please note that the arguments are not limited to just two but depends on the task to solve.

<img src="../images/ufunc.png" />
<div style="margin-left:350px">Figure 6.0   Ufuncs definition</div>

**Task 2**- Addition of two squares:  $a^2$+ $b^2$= $(a-b)^2$ + 2ab.

**Solution**:
```python
from numba import vectorize
import numpy as np

@vectorize(['float32(float32, float32)'], target='cuda')
def additionOfSquares(a, b):
    return (a - b)**2 + (2*a*b)

N = 10000
# prepare the input
A = np.arange(N , dtype=np.float32)
B = np.arange(N, dtype=np.float32)

# calling ufuncs
C = additionOfSquares(A, B)
print(C.reshape(100,100)) # print result

#expected result:
...
[1.8818000e+08 1.8821880e+08 1.8825760e+08 ... 1.9196242e+08 1.9200160e+08 1.9204080e+08]
[1.9208000e+08 1.9211920e+08 1.9215840e+08 ... 1.9590122e+08 1.9594080e+08 1.9598040e+08]
[1.9602000e+08 1.9605960e+08 1.9609920e+08 ... 1.9988002e+08 1.9992000e+08 1.9996000e+08]]
```
---
**Exercise 3**: Write a CUDA ufunc to solve difference of two squares $a^2$- $b^2$ = (a-b)(a+b)

---

In [ ]:
#write Ufunc here


N = 10000
# prepare the input
A = np.arange(N , dtype=np.float32)
B = np.arange(N, dtype=np.float32)

# call Ufunc here


The CUDA device functions can only be invoked from a Kernel function (not from Host) and would return value like Python functions. The device function is usually placed before the CUDA ufunc kernel otherwise a call to it may not be visible in the ufunc kernel. The attributes `device=True` and `inline=true` indicates `device_ufunc` as a device function. Solution for addition of squares is be rewritten as follows:

```python
from numba import vectorize
import numpy as np

@cuda.jit('float32(float32,float32)', device=True, inline=True)
def device_ufunc(a,b):
   return (a - b)**2

@vectorize(['float32(float32, float32)'], target='cuda')
def additionOfSquares(a, b):
    return device_ufunc(a,b) + (2*a*b)
```


## Atomic Operation

Atomic operation is required when multiple threads attempt to modify a common portion of the memory. Typical example includes simultaneous withdrawal from a bank account through ATM machines or many threads modifying a particular index of an array based on certain condition(s). In parallel execution, atomic operation helps eliminate race conditions that often occur in share resources. List of presently implemented atomic operations supported in Numba are:

    - cuda.atomic.add(array, index, value)
    - cuda.atomic.min(array, index, value)
	- cuda.atomic.max(array, index, value)
	- cuda.atomic.nanmax(array, index, value)
	- cuda.atomic.nanmin(array, index, value)
    - cuda.atomic.compare_and_swap(array, old_value, current_value)
	- cuda.atomic.sub(array, index, value)

Complete list can be found here: https://numba.pydata.org/numba-doc/dev/cuda/intrinsics.html#

In the cell below are two examples that sum elements of an array together in parallel. The first uses atomic operation approach and gives correct answer while the other does not, therefore gives incorrect result.  


In [ ]:
# Task ==> sum of an array: [1,2,3,4,5,6,7,8,9,10] in parallel
# Note that threads are executed randomly

import numba.cuda as cuda
import numpy as np

# atomic operation example 
size = 10
nthread = 10
@cuda.jit()
def add_atomic(my_array, total):
   tid = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
   cuda.atomic.add(total,0, my_array[tid])

my_array = np.array([1,2,3,4,5,6,7,8,9,10], dtype=np.int32)
total = np.zeros(1, dtype=np.int32)
nblock = int(size / nthread)
add_atomic[nblock, nthread](my_array, total)
print("Atomic:", total)

######################################################################################
# Non-atomic operation example  
size = 10
nthread = 10
@cuda.jit()
def add_non_atomic(my_array, total):
   tid = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
   total[0] += my_array[tid]
   

my_array = np.array([1,2,3,4,5,6,7,8,9,10], dtype=np.int32)
total = np.zeros(1, dtype=np.int32)
nblock = int(size / nthread)
add_non_atomic[nblock, nthread](my_array, total)
print("Non atomic: ", total)



In the atomic operation sample code, `cuda.to_device()` and `copy_to_host()` were purposely not used in order to show that NumPy arrays are visible from Host to Device and vice versa.  

---
**Exercise 4**: Write a Numba program (Host & Device) that counts prime numbers between `1 – 1,000,000`. The serial function is written for you to serve as a guide to write the corresponding Numba Kernel `count_prime_nos` and complete the rest of the Host code.

---

```python 

from numba import njit
import numpy as np

N = 1000000

@njit()
def prime_nos_count_serial(num, counter):
    for i in num:
        if i > 1:
            track = 0
            for j in range(2, i):
                if (i % j) == 0:
                    track += 1
                    break
            if track == 0:
                counter[0] += 1
                #print(i)
    return counter

```

In [ ]:
import numba.cuda as cuda
import numpy as np
import math

#kernel function
@cuda.jit()
def count_prime_nos(num_list, counter):
    ##complete the code 
    
   cuda.atomic.add(counter, 0, 1)

   #counter[0] += 1


    
#input data initialized on the Host
counter = np.zeros(1, dtype=np.int32)
num_list = np.arange(N, dtype=np.int32)
##prime_nos_count_serial(num_list, counter)


#set block and grid size and call kernel
threads_in_block = 512
##complete the rest


print(counter)

################# expected output #########
# [78498]
###########################################

## Summary

<img src="../images/numba_summary1.png"/>


---

## Lab Task

In this section, you are expected to click on the **Serial Code Lab Assignment** link and proceed to Lab 2. In this lab you will find three python serial code functions. You are required to revise the **pair_gpu** function to run on the GPU, and likewise do a few modifications within the **main** function.

## <div style="text-align:center; color:#FF0000; border:3px solid red;height:80px;"> <b><br/> [Serial Code Lab Assignment](serial_RDF.ipynb) </b> </div>

---

## Post-Lab Summary

If you would like to download this lab for later viewing, we recommend you go to your browser's File menu (not the Jupyter notebook file menu) and save the complete web page. This will ensure the images are copied as well. You can also execute the following cell block to create a zip-file of the files you've been working on and download it with the link below.



In [ ]:
%%bash
cd ..
rm -f nways_files.zip
zip -r nways_files.zip *


**After** executing the above zip command, you should be able to download and save the zip file by holding down <mark>Shift</mark> and <mark>Right-Clicking</mark> [Here](../nways_files.zip).

**IMPORTANT**: Please click on **HOME** to go back to the main notebook for *N ways of GPU programming for MD* code.

---

# <p style="text-align:center;border:3px; border-style:solid; border-color:#FF0000  ; padding: 1em"> <a href=../../../nways_MD_start_python.ipynb>HOME</a></p>

---


# Links and Resources

[NVIDIA Nsight System](https://docs.nvidia.com/nsight-systems/)

[NVIDIA CUDA Toolkit](https://developer.nvidia.com/cuda-downloads)

**NOTE**: To be able to see the Nsight System profiler output, please download the latest version Nsight System from [here](https://developer.nvidia.com/nsight-systems).

Don't forget to check out additional [OpenACC Resources](https://www.openacc.org/resources) and join our [OpenACC Slack Channel](https://www.openacc.org/community#slack) to share your experience and get more help from the community.

---


## References

- Numba Documentation, Release 0.52.0-py3.7-linux-x86_64.egg, Anaconda, Nov 30, 2020.
- Bhaumik Vaidya, Hands-On GPU-Accelerated Computer Vision with OpenCV and CUDA, Packt Publishing, 2018.
- https://docs.nvidia.com/cuda/cuda-c-programming-guide/


--- 

## Licensing 

This material is released by OpenACC-Standard.org, in collaboration with NVIDIA Corporation, under the Creative Commons Attribution 4.0  International (CC BY 4.0).